#### 思路：

（1）构建农作物种类及其管理方式的所有可能组合

（2）然后结合农作物种类匹配好气候因素观测值

（3）最后对于每一个组合去结合每个网格的土壤属性数据去估算该网格的排放因子EF值

其中(1)和(2)的数据要合并在一起构成数据集。


In [1]:
import os
import time
import pandas as pd
import numpy as np
from itertools import product

In [2]:
# 定义5个属性变量及其取值
var1 = ['AN', 'EEF', 'Manure', 'Others', 'U']
var2 = ['DPM', 'Mix', 'SBC']
var3 = [1, 2]
var4 = ['CT', 'NT']
var5 = ['Maize', 'Rice', 'Wheat']

# 生成所有可能的组合（笛卡尔积）
all_combinations = list(product(var1, var2, var3, var4, var5))

# 转换为DataFrame，指定列名
df = pd.DataFrame(all_combinations,
                  columns=['Fertilizer type','Nitrogen placement', 'Fertilizer application time', 'Soil tillage practices', 'Crop type'])

# df.to_csv('农作物管理组合.csv')

In [3]:
# 读取土壤数据(不需要将grid_num值补充完整)
df_soil = pd.read_csv('2018年土壤特征(汇总).csv')
# df_soil = df_soil.merge(pd.DataFrame({'grid_num':range(1, 360*720+1)}), on='grid_num', how='right')

In [4]:
# 读取从作物的水输入和平均温度基础数据
df_tem = pd.read_csv('2018年各种农作物平均温度(汇总).csv')
df_water = pd.read_csv('2018年各种农作物水输入总量(汇总).csv')
# df_water = df_water.merge(pd.DataFrame({'grid_num':range(1, 360*720+1)}), on='grid_num', how='right')

In [5]:
# 记录开始时间
start_time = time.time()  # 返回当前时间的时间戳（秒）

result_dict = {} # 存储所有的结果
# 遍历土壤网格中的数据作为最后的结果
for j in df_soil['grid_num']:

    # 储存第一季农作物数据    
    df_crop1_grids = df
    Water_input1 = []
    tems1 = []
    
    # 储存第二季农作物数据 
    df_crop2_grids = df
    Water_input2 = []
    tems2 = []
    
    for k in range(len(df_crop1_grids)):
        
        if df_crop1_grids.loc[k,'Crop type'] == 'Rice':
            Water_input1.append(round(df_water.loc[j,'water_input_Rice'],1)) #网格grid_num=1
            tems1.append(round(df_tem.loc[j,'average_tem_Rice(grids)'],1))
        
        if df_crop1_grids.loc[k,'Crop type'] == 'Maize':
            Water_input1.append(round(df_water.loc[j,'water_input_Maize'],1))
            tems1.append(round(df_tem.loc[j,'average_tem_Maize(grids)'],1))
    
        if df_crop1_grids.loc[k,'Crop type'] == 'Wheat':
            Water_input1.append(round(df_water.loc[j,'water_input_Wheat'],1))
            tems1.append(round(df_tem.loc[j,'average_tem_Wheat(grids)'],1))
    
    for k in range(len(df_crop2_grids)):
        
        if df_crop2_grids.loc[k,'Crop type'] == 'Rice':
            Water_input2.append(round(df_water.loc[j,'water_input_Rice.2'],1))  #网格grid_num=1
            tems2.append(round(df_tem.loc[j,'average_tem_Rice.2(grids)'],1))
        
        if df_crop2_grids.loc[k,'Crop type'] == 'Maize':
            Water_input2.append(round(df_water.loc[j,'water_input_Maize.2'],1))
            tems2.append(round(df_tem.loc[j,'average_tem_Maize.2(grids)'],1))
    
        if df_crop2_grids.loc[k,'Crop type'] == 'Wheat':
            Water_input2.append(round(df_water.loc[j,'water_input_Wheat.2'],1))
            tems2.append(round(df_tem.loc[j,'average_tem_Wheat.2(grids)'],1))
    
    df_crop1_grids['Water input'] = Water_input1
    df_crop1_grids['Tem'] = tems1
    df_copy1 = pd.concat([df_soil[df_soil['grid_num']==j]] * len(df_crop1_grids), ignore_index=True).drop(columns=['grid_num'])
    merged_df1 = pd.concat([df_crop1_grids, df_copy1], axis=1)

    df_crop2_grids['Water input'] = Water_input2
    df_crop2_grids['Tem'] = tems2
    df_copy2 = pd.concat([df_soil[df_soil['grid_num']==j]] * len(df_crop2_grids), ignore_index=True).drop(columns=['grid_num'])
    merged_df2 = pd.concat([df_crop2_grids, df_copy2], axis=1)
    
    result = pd.concat([merged_df1, merged_df2], axis=0, ignore_index=True)
    result.to_csv(f'./compile_data_result/grid_num_{j}.csv', index=False)
    result_dict[f'grid_num_{j}'] = result

# 记录结束时间
end_time = time.time()

# 计算耗时
elapsed_time = end_time - start_time
print(f"耗时: {elapsed_time/60:.2f} 分")

耗时: 10.17 分
